In [1]:
from pyspark import SparkContext, SparkConf
conf = SparkConf().setAppName("appName")
sc = SparkContext(conf=conf)
from pyspark.sql import SQLContext, HiveContext
sqlContext = SQLContext(sc)
#sc.version
#sqlContext
sc

<SparkContext master=local[*] appName=appName>

In [18]:
#from pyspark.streaming.kafka import KafkaUtils
#kafka = KafkaUtils.createStream()

In [29]:
from kafka import KafkaConsumer,TopicPartition
import json

topicName="spyData"
consumer = KafkaConsumer(bootstrap_servers=['localhost:9092'])
tp = TopicPartition(topicName,0)
#register to the topic
consumer.assign([tp])

# obtain the last offset value
consumer.seek_to_end(tp)
lastOffset = consumer.position(tp)

consumer.seek_to_beginning(tp)
for message in consumer:
    print(message)
    if message.offset == lastOffset - 1:
        break

ConsumerRecord(topic='spyData', partition=0, offset=0, timestamp=1593018648146, timestamp_type=0, key=None, value=b'{"stream":"T.SPY","data":{"ev":"T","T":"SPY","i":"3472328506985769590","x":15,"p":304.965,"s":100,"t":1593018558069000000,"c":null,"z":1}}', headers=[], checksum=None, serialized_key_size=-1, serialized_value_size=138, serialized_header_size=-1)
ConsumerRecord(topic='spyData', partition=0, offset=1, timestamp=1593018648521, timestamp_type=0, key=None, value=b'{"stream":"T.SPY","data":{"ev":"T","T":"SPY","i":"3472328506985899607","x":15,"p":304.96,"s":10,"t":1593018558452000000,"c":[37],"z":1}}', headers=[], checksum=None, serialized_key_size=-1, serialized_value_size=136, serialized_header_size=-1)
ConsumerRecord(topic='spyData', partition=0, offset=2, timestamp=1593018736754, timestamp_type=0, key=None, value=b'{"stream":"T.SPY","data":{"ev":"T","T":"SPY","i":"52983525033804","x":9,"p":305.23,"s":2,"t":1593018646618000000,"c":[37],"z":1}}', headers=[], checksum=None, ser

In [4]:
streamingInputData = sqlContext\
                    .readStream\
                    .format("kafka")\
                    .option("kafka.bootstrap.servers","localhost:9092")\
                    .option("subscribe","spyData")\
                    .load()
#streamingInputData.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)")

In [5]:
streamingInputData.printSchema

<bound method DataFrame.printSchema of DataFrame[key: binary, value: binary, topic: string, partition: int, offset: bigint, timestamp: timestamp, timestampType: int]>

In [6]:
import json

In [ ]:
selectDF = streamingInputData.select(get_json_object($"value").cast)